In [9]:
!pip install flask-ngrok

# Train MLP

In [10]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

### preprocess data

In [11]:
# load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
  y = np.load(f)
  

# select features

# OverallQual 10
# GrLivArea 10
# GarageCars 9
# GarageArea 8
# TotalBsmtSF 7
# 1stFlrSF 6
# FullBath 5
# LotShape Reg

X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

### Training

In [12]:
model = keras.Sequential(
    [
        keras.Input(shape=scaled_X.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
    ]
)
model.compile(loss="mse", optimizer="adam")

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.fit(scaled_X, scaled_y,
          batch_size=2, epochs=150,
          callbacks=[early_stopping_callback], validation_split=0.05)

Epoch 1/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0051 - val_loss: 0.0090
Epoch 2/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0037 - val_loss: 0.0017
Epoch 3/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0031 - val_loss: 0.0062
Epoch 4/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0016
Epoch 5/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0021
Epoch 6/150
694/694 [==============================] - 1s 965us/step - loss: 0.0025 - val_loss: 0.0027
Epoch 7/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0026 - val_loss: 0.0014
Epoch 8/150
694/694 [==============================] - 1s 977us/step - loss: 0.0024 - val_loss: 0.0018
Epoch 9/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0023 - val_loss: 0.0012
Epoch 10/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - va

In [13]:
pred = model.predict(scaled_X[:1])
pred = y_min_max_scaler.inverse_transform(pred)

### Save MLP model

In [14]:
model.save("mlp_v0.1.h5")

In [15]:
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

In [16]:
pred = reconstructed_model.predict(scaled_X[:1]) # 0 - 1
pred = y_min_max_scaler.inverse_transform(pred)

In [17]:
pred

array([[189284.5]], dtype=float32)

# Flask server

In [19]:
# Read data (v)
# Preprocess data
# Model prediction (v)
# Return predict

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
  y = np.load(f)

# OverallQual 10
# GrLivArea 10
# GarageCars 9
# GarageArea 8
# TotalBsmtSF 7
# 1stFlrSF 6
# FullBath 5
# LotShape Reg
X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

# 우리가 모델을 띄울 때 항상 학습을 시키지 않을 것이기 때문에
# 모델과 min max scaler는 Flask 서버가 실행될 때 바로 로딩할 수 있는 상태가 되어야 한다.
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)

# load model
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")


# run server
app = Flask(__name__, template_folder='/content')
run_with_ngrok(app)

def preprocess_data(data):
  # TODO: 전처리

  '''
  Dictionary --> np array (1, 8)

  OverallQual 2
  GrLivArea 5000
  GarageCars 2
  GarageArea 480
  TotalBsmtSF 991
  1stFlrSF 1087
  FullBath 2
  LotShape IR3
  '''
  
  # Scale normalizaion
  X = []  # <-- OverallQual, GrLivArea, ..., LotShape
  for k, v in data.items():
    if k == 'LotShape':
      if v == 'Reg':
        X.append(4)
      elif v == 'IR1':
        X.append(3)
      elif v == 'IR2':
        X.append(2)
      elif v == 'IR3':
        X.append(1)
    else:
      X.append(float(v))
  
  # X = [2, 5000, 2, ..., 3]
  X = np.array(X) # (8, )
  X = X.reshape((1, -1)) # (1, 8)

  # min max scaling
  scaled_X = x_min_max_scaler.transform(X)

  #return np.zeros((1, 8)) # dummy data
  return scaled_X

@app.route("/")
def predict():
  # return "<h1>This is your Flask server.</h1>"
  return render_template("submit_form.html")

@app.route("/result", methods=['POST'])
def result():
  data = request.form # User data

  message = ""
  message += "<h1>House Price</h1>"

  for k, v in data.items():
    print(k, v)
    message += k + ": " + v + "</br>"
  
  # 데이터 전처리
  X = preprocess_data(data) # User data --> (1, 8) array

  pred = reconstructed_model.predict(X)
  pred = y_min_max_scaler.inverse_transform(pred)
  # array (1, 1) --> string

  message += "</br>"
  message += "Predicted price: " + str(pred[0][0])

  return message

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8c7f94f8e450.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Aug/2021 04:16:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 04:16:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Aug/2021 04:16:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 04:16:15] "POST /result HTTP/1.1" 200 -


OverallQual 6
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1


127.0.0.1 - - [11/Aug/2021 04:16:17] "GET /result HTTP/1.1" 405 -
127.0.0.1 - - [11/Aug/2021 04:16:33] "POST /result HTTP/1.1" 200 -


OverallQual 10
GrLivArea 1508
GarageCars 0
GarageArea 480
TotalBsmtSF 970
1stFlrSF 1114
FullBath 2
LotShape IR2
